In [1]:
import os, sys
import pandas as pd
sys.path.append(os.path.abspath(os.path.join("../dashboard")))
sys.path.append(os.path.abspath(os.path.join("")))

from db_setup import DBHandler

In [2]:
processed_tweets = pd.read_csv('../data/processed_tweet_data.csv')
missing_values_count = processed_tweets.isnull().sum()
# missing_df=pd.DataFrame(missing_values_count)
missing_df=pd.DataFrame(missing_values_count).rename_axis('missing_col').reset_index()
missing_df.rename(columns={0:"count"},inplace=True)
missing_df

,missing_col,count
0,created_at,0
1,source,0
2,original_text,0
3,polarity,0
4,subjectivity,0
5,lang,0
6,favorite_count,0
7,retweet_count,0
8,original_author,0
9,followers_count,0


In [3]:
import re

def remove_empty_str(word : str):
    """Process hashtag function.
    Input:
        hashtag: a string containing a hashtag
    Output:
        word: cleaned hashtag

    """
    # remove empty spaces from a word
    word = word.strip()
    
    return word
def extract_text_from_hashtag(hashtag : str):
    """Process hashtag function.
    Input:
        hashtag: a string containing a hashtag
    Output:
        word: cleaned hashtag

    """
    # remove array characters /[\[\]']+/g
    hashtag = hashtag.replace("[","")
    hashtag = hashtag.replace("]","")
    hashtag = hashtag.replace("'","")

    return hashtag

In [4]:
cleaned_tweets = pd.read_csv('../data/cleaned_tweet_data.csv')
flattened_words_df = pd.DataFrame(
    [word for words_list in cleaned_tweets.clean_text
    for word in words_list.split(' ')],
    columns=['word'])

flattened_words_df.word=flattened_words_df.word.apply(remove_empty_str)
flattened_words_df.drop(flattened_words_df[flattened_words_df['word']==""].index,axis=0,inplace=True)
# freq_words_df=pd.DataFrame(flattened_words_df.word.value_counts()[:20])
freq_words_df=pd.DataFrame(flattened_words_df.word.value_counts()[:20]).rename_axis('word_text').reset_index()


In [5]:
freq_words_df.rename(columns={"word":"count"},inplace=True)
freq_words_df

,word_text,count
0,the,10900
1,to,9896
2,is,6580
3,you,5823
4,for,5618
5,and,5174
6,of,5018
7,in,4406
8,a,4234
9,are,3847


In [6]:
# remove empty hashtags
cleaned_tweets.drop(cleaned_tweets[cleaned_tweets['hashtags']=="[]"].index,inplace=True)
flattened_hashtags_df = pd.DataFrame(
    [hashtag for hashtags_list in cleaned_tweets.hashtags
    for hashtag in hashtags_list.split(',')],
    columns=['hashtag'])

flattened_hashtags_df.drop(flattened_hashtags_df[flattened_hashtags_df['hashtag'] == ""].index, inplace=True)
flattened_hashtags_df.reset_index(drop=True, inplace=True)
flattened_hashtags_df['hashtag']=flattened_hashtags_df.hashtag.apply(extract_text_from_hashtag)
freq_hashtag_df=pd.DataFrame(flattened_hashtags_df.hashtag.value_counts()[:20]).rename_axis('hashtag_name').reset_index()

In [7]:
freq_hashtag_df.rename(columns={"hashtag":"count"},inplace=True)
freq_hashtag_df

,hashtag_name,count
0,Taiwan,10557
1,Pelosi,5038
2,Taiwan,4248
3,China,2919
4,NancyPelosi,2260
5,Chinese,2124
6,Pelosi,1309
7,helicopter,985
8,sabinus,916
9,BBNaija,878


In [8]:
db_obj=DBHandler()
db_obj.create_table("tweet_data","missing_data.sql")
db_obj.create_table("tweet_data","freq_words.sql")
db_obj.create_table("tweet_data","freq_hash.sql")

In [9]:

db_obj.populate_table(missing_df,"missing_data","tweet_data")
db_obj.populate_table(freq_words_df,"freq_words","tweet_data")
db_obj.populate_table(freq_hashtag_df,"freq_hashtags","tweet_data")